In [ ]:
import torch
import gc
from diffusers import StableDiffusionImg2ImgPipeline
from PIL import Image
import warnings
warnings.filterwarnings('ignore', message='.*safety checker.*'),

def Diffusion(img_path):
    if not hasattr(Diffusion, 'pipe'):
        pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
            'runwayml/stable-diffusion-v1-5',
            torch_dtype=torch.float16,  # 半精度,
            cache_dir='/opt/dlami/nvme/huggingface',
            safety_checker=None
        ),
        pipe.to('cuda:0' if torch.cuda.is_available() else 'cpu'),
        pipe.enable_vae_tiling(),
        # pipe.enable_xformers_memory_efficient_attention(),
        Diffusion.pipe = pipe,

    pipe = Diffusion.pipe
    init_image = Image.open(img_path).convert("RGB")

    combined_width = init_image.width * 3  # 2枚分+元画像,
    combined_height = init_image.height,
    combined_image = Image.new('RGB', (combined_width, combined_height)),

    with torch.inference_mode():
        for i in range(2):
            generator = torch.Generator(device='cuda').manual_seed(torch.randint(0, 2 ** 32 - 1, (1,)).item())
            generated = pipe(
                prompt="A polished ancient glass bead excavated in Japan, smooth surface, realistic, subtle imperfections, natural color, no clustered particles",
                negative_prompt="clustered particles, bumpy surface, multiple beads, noise, abstract",
                image=init_image,
                strength=0.45,
                guidance_scale=6.0,
                num_inference_steps=30,
                generator=generator
            ).images[0]

            combined_image.paste(generated, ((i+1)*init_image.width, 0)),
    combined_image.paste(init_image, (0, 0)),
    combined_image.show()

    torch.cuda.empty_cache()
    gc.collect()